In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tqdm import tqdm
import os
import cv2

import scienceplots


plt.style.use(['science', 'ieee'])
plt.rcParams.update({"font.family": "sans", "font.serif": [
                    "cm"], "mathtext.fontset": "cm", "font.size": 22})

In [ ]:
W_list = np.load(f'./data/W_constant.npy')
S_list = np.load(f'./data/S_constant.npy')
time_list = np.load('./data/time_list_constant.npy')
model = "constant"
output_dir = './videos/histogram_videos_fps3'
os.makedirs(output_dir, exist_ok=True)

# Placeholder for filenames to be generated
filenames = []

xmax = max(np.max(S_list), np.max(W_list))
xmin = min(np.min(S_list), np.min(W_list))

# Iterate over the list of S and W matrices to plot histograms
for i in tqdm(range(len(S_list))):
    # Flatten the arrays
    S_flat = S_list[i].reshape(-1)
    W_flat = W_list[i].reshape(-1)

    # Create the histogram plot
    plt.figure(figsize=(12, 6))
    sns.histplot(S_flat, bins=128, kde=False, color='blue', stat='density', linewidth=0, alpha=0.5, label='S')
    sns.histplot(W_flat, bins=128, kde=False, color='salmon', stat='density', linewidth=0, alpha=0.5, label='W')

    # Fit and plot the normal distribution for S
    mu_s, std_s = stats.norm.fit(S_flat)
    x = np.linspace(xmin, xmax, 100)
    p_s = stats.norm.pdf(x, mu_s, std_s)
    plt.plot(x, p_s, 'b--', linewidth=1, label=f'S Fit ($\mu$={mu_s:.2f}, $\sigma$={std_s:.2f})')

    # Fit and plot the normal distribution for W
    mu_w, std_w = stats.norm.fit(W_flat)
    p_w = stats.norm.pdf(x, mu_w, std_w)
    plt.plot(x, p_w, 'r--', linewidth=1, label=f'W Fit ($\mu$={mu_w:.2f}, $\sigma$={std_w:.2f})')

    # Format the time string with padding
    str_time = f'{round(float(time_list[i]), 4):0>8.3f}'

    # Set the title with the epoch, time, and fit results
    plt.title(f"Epoch:{str(i).zfill(2)}, Time:{str_time} seconds\nFIT RESULTS: S $\mu$ = {mu_s:.2f}, S $\sigma$ = {std_s:.2f}, W $\mu$ = {mu_w:.2f}, W $\sigma$ = {std_w:.2f}")
    plt.xlabel('VALUE')
    plt.ylabel('DENSITY')
    plt.grid(True)
    plt.legend()
    plt.ylim(0, 17)

    # Save each frame as a temporary PNG file
    filename = f'{output_dir}/S_W_{model}_ep_{str(i).zfill(2)}.png'
    plt.savefig(filename)
    plt.close()
    # plt.show()
    # Append the filename to the list
    filenames.append(filename)

# Output the first filename to check if the image is saved correctly
filenames[0]


In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'x264' if you have it
fps = 3  # Frames per second

# Determine the resolution of the video
# Read the first image to determine the frame size
first_image_path = filenames[0]
frame = cv2.imread(first_image_path)
height, width, layers = frame.shape
frame_size = (width, height)

# Initialize the video writer
video_filename = f'./S_W_histograms.mp4'  # Path where the MP4 video will be saved
out = cv2.VideoWriter(video_filename, fourcc, fps, frame_size)

for filename in tqdm(filenames):
    frame = cv2.imread(filename)
    if frame is None:
        raise Exception
    out.write(frame)  # Write out the frame

out.release()

video_filename

In [ ]:
cv2.imread(filenames[99])